In [ ]:
from google.cloud import bigquery
import pandas as pd

from google.oauth2 import service_account

key_path = "crunchbase-startups-b791b011b22c.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

sql = """
    SELECT *
    FROM `crunchbase-startups.startups.join_table`
"""

df = client.query(sql).to_dataframe()

DefaultCredentialsError: File C:\Users\arjun\AppData\Roaming\gcloud\application_default_credentials.json was not found.